# **Homework 1: Linear Regression**

本次目标：由前 9 个小时的 18 个 features (包含 PM2.5)预测的 10 个小时的 PM2.5。<!-- 可以参考 <link> 获知更细项的作业说明。-->

<!-- 首先，从 https://drive.google.com/open?id=1El0zvTkrSuqCTDcMpijXpADvJzZC2Jpa 将整个文件夹下载下来，并将下载下来的文件夹放到自己的 Google Drive（注意：上传到自己 Google Drive 的是文件夹 hw1-regression，而非压缩文件） -->


若有任何问题，欢迎来信至助教信箱 ntu-ml-2020spring-ta@googlegroups.com

# **Load 'train.csv'**
train.csv 的资料为 12 个月中，每个月取 20 天，每天 24 小时的数据(每小时数据有 18 个 features)。

In [ ]:
import sys
import pandas as pd
import numpy as np
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
# 读取事先存放好的Training data
data = pd.read_csv('./train.csv', encoding = 'big5')
data

# **Preprocessing** 
取需要的数值部分，将 'RAINFALL' 字段全部补 0。
另外，如果要在 colab 重复这段程序代码的执行，请从头开始执行(把上面的都重新跑一次)，以避免跑出不是自己要的结果（若自己写程序不会遇到，但 colab 重复跑这段会一直往下取资料。意即第一次取原本数据的第三栏之后的数据，第二次取第一次取的数据掉三栏之后的数据，...）。

In [ ]:
# 我们只需要第三列以后的data
data = data.iloc[:,3:]
# 将NR的数据全部置为0
data[data == 'NR'] = 0
# 将dataframe幻化成numpy
raw_data = data.to_numpy()

#查看raw_data
raw_data

# **Extract Features (1)**
![图片说明](https://drive.google.com/uc?id=1LyaqD4ojX07oe5oDzPO99l9ts5NRyArH)
![图片说明](https://drive.google.com/uc?id=1ZroBarcnlsr85gibeqEF-MtY13xJTG47)

将原始 4320(=18 * 12 * 20) * 18 的资料依照每个月分重组成 12 个 18 (features) * 480 (hours) 的资料。

In [ ]:
# 现在要开始处理Training data
# 由于提供的数据是每个月前20天(每天24小时)的资料(共18种类型)，因此每个月的这20*24=480个小时的时间是连续的，每10个小时可以作为一笔data，480小时共471笔data
# 由于不同月份之间的时间是不连续的，因此先把所有的资料按照月份分开
month_data = {}
for month in range(12):
    # 每个月共20*24=480份数据，共18种空气成分，每一种成分每月都有480份数据，因此初始化一个18*480的array
    sample = np.empty([18, 480])
    for day in range(20):
        # sample中加入第day天的数据，由于每天都有24份数据共24列,故列的范围是24*day~24*(day+1)；选择加入sample的是第20×month+day这一天的数据，由于每天有18种大气成分的数据共18行，因此行的范围是18*(20*month+day)~18*(20*month+day+1)
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    # 把这个月的data放进month_data[month]里去
    month_data[month] = sample

# 查看第一个月的数据，并借此查看数据的结构：每一行都是一种大气成分在24*20个连续小时内的值
month_data[0]

# **Extract Features (2)**
![alt text](https://drive.google.com/uc?id=1wKoPuaRHoX682LMiBgIoOP4PDyNKsJLK)
![alt text](https://drive.google.com/uc?id=1FRWWiXQ-Qh0i9tyx0LiugHYF_xDdkhLN)

每个月会有 480hrs，每 9 小时形成一个 data，每个月会有 471 个 data，故总资料数为 471 * 12 笔，而每笔 data 有 9 * 18 的 features (一小时 18 个 features * 9 小时)。

对应的 target 则有 471 * 12 个(第 10 个小时的 PM2.5)

In [ ]:
# 这个时候我们就要在12段连续的时间里每10个小时取出一笔Training data
# 每个月连续时间有20*24=480h，每10h作为一笔data，共可以分为471笔data；一共12个月，因此共12*471笔data
# 每一笔data，前9h为input，最后1h的PM2.5为output，因此input是一个18*9的矩阵，如果把它摊平就是一个18*9的feature vector

# x是input，共12*471笔data，每个input是一个18*9的feature vector
x = np.empty([12 * 471, 18 * 9], dtype = float)
# y是output，共12*471笔data，每个output是第10个小时的第9种大气成分PM2.5，因此只有1维
y = np.empty([12 * 471, 1], dtype = float)
# 取出input和output
# 每个月份共471笔data，依次遍历即可
 # 用一个18行(18种物质),10列(10个小时的数据，包括input 18行(18种物质),9列(9个小时) 变成9 * 18 维的一行
 # 和output18行(18种物质),1列(1个小时)) 来存放每一笔Training data
for month in range(12):
    for day in range(20):
        for hour in range(24):
            # 19天 15h 代表本月取完，开始取下个月
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
             # month_data前9列的作为input的数据(18行9列)用reshape平摊到一个18*9的行向量上
             # month_data的第10列作为output，只有第10行的PM2.5值是有用的
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

# **Normalize (1)**


In [ ]:
# 这个时候我们已经有了Training data的x和y，但由于feature的每一个dimension大小范围都是不一样的，因此还要对其做feature scale
# 求出期望mean和标准差std，利用公式x'=(x-u)/σ来使同一列的数据同时满足同一个分布
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
print(mean_x.shape)
print(std_x.shape)
x

# **Split Training Data Into "train_set" and "validation_set"**
这部分是针对作业中 report 的第二题、第三题做的简单示范，以生成比较中用来训练的 train_set 和不会被放入训练、只是用来验证的 validation_set。

In [ ]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

# **Training**
![alt text](https://drive.google.com/uc?id=1xIXvqZ4EGgmxrp7c9r0LOVbcvd4d9H4N)
![alt text](https://drive.google.com/uc?id=1S42g06ON5oJlV2f9RukxawjbE4NpsaB6)
![alt text](https://drive.google.com/uc?id=1BbXu-oPB9EZBHDQ12YCkYqtyAIil3bGj)

(和上图不同处: 下面的 code 采用 Root Mean Square Error)

因为常数项的存在，所以 dimension (dim) 需要多加一栏；eps 项是避免 adagrad 的分母为 0 而加的极小数值。

每一个 dimension (dim) 会对应到各自的 gradient, weight (w)，透过一次次的 iteration (iter_time) 学习。

In [ ]:
# 现在开始尝试简单的linear Regression: y = w_1*x_1+w_2*x_2+...+b
# 由于常数项的存在，dimension需要在18*9的基础上再加上一维，初始化每一维对应的weight为0，并进行gradient descent，经过多次iteration来学习
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
# 利用np.concatenate函数在原先的x上加上一个常数的feature=1,用一个全1的列向量，并使用axis=1来横向拼接
learning_rate = 100
iter_time = 1000
adagrad = np.zeros([dim, 1])
# adagrad表达式为w'=w-lr*gd/sqrt(Σgd^2))，为了避免分母为0，加上一个很小的eps
eps = 0.0000000001
# loss function = (x*w-y)^2, loss对w求导为 2*x*(x*w-y),即gradient
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    # 更新w,注意这里gradient、adagrad都是列向量，加减乘除的用法跟常数一样
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

# **Testing**
![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

加载 test data，并且以相似于训练数据预先处理和特征萃取的方式处理，使 test data 形成 240 个维度为 18 * 9 + 1 的资料。

In [ ]:
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

# **Prediction**
说明图同上

![alt text](https://drive.google.com/uc?id=1165ETzZyE6HStqKvgR0gKrJwgFLK6-CW)

有了 weight 和测试数据即可预测 target。

In [ ]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

# **Save Prediction to CSV File**


In [ ]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

相关 reference 可以参考:

Adagrad :
https://youtu.be/yKKNr-QKz2Q?list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&t=705 

RMSprop : 
https://www.youtube.com/watch?v=5Yt-obwvMHI 

Adam
https://www.youtube.com/watch?v=JXQT_vxqwIs 


以上 print 的部分主要是为了看一下资料和结果的呈现，拿掉也无妨。另外，在自己的 linux 系统，可以将档案写死的的部分换成 sys.argv 的使用 (可在 terminal 自行输入档案和档案位置)。

最后，可以藉由调整 learning rate、iter_time (iteration 次数)、取用 features 的多寡(取几个小时，取哪些特征字段)，甚至是不同的 model 来超越 baseline。

Report 的问题模板请参照 : https://docs.google.com/document/d/1s84RXs2AEgZr54WCK9IgZrfTF-6B1td-AlKR9oqYa4g/edit